In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, TimeSeriesSplit

idle_time_data = pd.read_csv('../data/final_df_points_18_21.csv')

TargetVariable = ['idle_time']
Predictors = ['bike_id', 'lat', 'lng', 'temp', 'rain', 'snow', 'dt_start', 'hex_enc', 'start_min', 'month', 'day']

X = idle_time_data[Predictors].values
y = idle_time_data[TargetVariable].values

## Scale Data
#PredictorScaler = StandardScaler()
#PredictorScalerFit = PredictorScaler.fit(X)
#X = PredictorScalerFit.transform(X)

#TargetScaler = StandardScaler()
#TargetScalerFit = TargetScaler.fit(y)
#y = TargetScalerFit.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)
# , random_state=42

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import zero_one_loss, accuracy_score

model=RandomForestRegressor()
print(model)
model.fit(X_train,y_train.ravel())
score_training = model.score(X_train, y_train.ravel())
score_validation = model.score(X_test, y_test.ravel())
print(score_validation)
print(score_training)

y_pred = model.predict(X_test)

In [6]:
from sklearn.metrics import mean_squared_error
acc = mean_squared_error(y_test.ravel(), y_pred.ravel())
acc

0.5206879544940034

In [16]:
from sklearn import metrics
import numpy as np
from sklearn.metrics import zero_one_loss, accuracy_score
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import wandb
import wandb

sweep_configuration_rfr = {
    "project": "RandomForestRegressor",
    "name": "RFR-sweep-inorder-split-scaled",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "random",
    "parameters": {
        "n_estimators": {
            "values": [int(x) for x in np.linspace(start = 20, stop = 200, num = 10)]
        },
        "criterion": {
            "values": ['squared_error','absolute_error']
        },
        "max_depth": {
            "values": [5,10,20,30,40,50,60,70,80,90,100,None]
        },
        "bootstrap": {
            "values": [True, False]
        },
        "max_features": {
            "values": ['auto', 'sqrt', 'log2']
        },
        "min_samples_leaf": {
            "values": [1, 2, 4, 10, 20, 50, 100, 200, 1000]
        },
        "min_samples_split": {
            "values": [2, 5, 10, 100, 200]
        }
    }
}

#[int(x) for x in np.linspace(start = 20, stop = 200, num = 10)]
#[5,10,20,30,40,50,60,70,80,90,100,None]

def my_train_func():
    wandb.init()

    _n_estimators = wandb.config.n_estimators
    _criterion = wandb.config.criterion
    _max_depth = wandb.config.max_depth
    _bootstrap = wandb.config.bootstrap
    _max_features = wandb.config.max_features
    _min_samples_leaf = wandb.config.min_samples_leaf
    _min_samples_split = wandb.config.min_samples_split

    model = RandomForestRegressor(n_estimators=_n_estimators,
                                   criterion=_criterion,
                                   max_depth=_max_depth,
                                   bootstrap=_bootstrap,
                                   max_features=_max_features,
                                   min_samples_leaf=_min_samples_leaf,
                                  min_samples_split=_min_samples_split,
                                  verbose=True,
                                  n_jobs=2
                                  )

    # fit and predict
    model.fit(X_train,y_train.ravel())
    y_pred = model.predict(X_test)

    score_training = model.score(X_train, y_train.ravel())
    score_validation = model.score(X_test, y_test.ravel())
    mse = metrics.mean_squared_error(y_test.ravel(),y_pred.ravel())

    wandb.log({"mse":mse})
    wandb.log({"score_training": score_training, "score_validation": score_validation})


# INIT SWEEP
sweep_id_rfc = wandb.sweep(sweep_configuration_rfr, project="RandomForestRegressor")
# RUN SWEEP
wandb.agent(sweep_id_rfc, function=my_train_func)


Create sweep with ID: qz4hu1q4
Sweep URL: https://wandb.ai/jonathanweske/RandomForestRegressor/sweeps/qz4hu1q4


wandb: Agent Starting Run: xskayjk6 with config:
wandb: 	bootstrap: True
wandb: 	criterion: absolute_error
wandb: 	max_depth: 70
wandb: 	max_features: auto
wandb: 	min_samples_leaf: 50
wandb: 	min_samples_split: 100
wandb: 	n_estimators: 80


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


wandb: Ctrl + C detected. Stopping sweep.


In [17]:
wandb.finish()

In [20]:
y_test.shape

(509432, 1)

In [3]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1,verbose=True)
print(model.get_params())
model.fit(X_train,y_train.ravel())
y_pred = model.predict(X_test)


{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': None, 'verbose': True, 'warm_start': False}


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 12.1min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    7.5s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   14.6s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   28.9s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  1.2min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   31.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   52.4s finished


NameError: name 'metrics' is not defined

In [4]:
from sklearn import metrics

score_training = model.score(X_train,y_train.ravel())
score_validation = model.score(X_test,y_test.ravel())
rmse = metrics.mean_squared_error(y_test.ravel(),y_pred.ravel())

print(score_training)
print(score_validation)
print(rmse)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   30.5s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  1.3min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   17.1s


0.9497707487159038
0.3480222287015543
0.5269138711566934


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   33.8s finished
